### Content
- import df ords_prods_merge
- aggregated mean of 'order number' grouped by 'department_id'
- create loyalty flag
- customer spending habit based on loyalty flag
- create spending flag
- order frequency flag

### 01 import Data 

In [ ]:
# importing lybaries

import pandas as pd
import numpy as np
import os

In [ ]:
# import data sets

path = r'/Users/lennartzeidler/Desktop/Uni/Instacart Basket Analysis 08.10.2022 (4)'

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_sale_performance.pkl'))

In [ ]:
ords_prods_merge.head()

In [ ]:
ords_prods_merge.shape

### 02/03 aggregated mean of 'order number' grouped by 'department_id'

In [ ]:
# first step goup data

ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

#### Results  
When taking a look at the departments that were included in this subset, we'll see that only 8 of the total 21 are. This indicates that information is leaking from the subset. The average numbers also vary a little.

### 04 create loyalty flag

In [ ]:
# create new column containing max order number of user

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# turning max row-display option off

pd.options.display.max_rows = None

In [ ]:
ords_prods_merge.head(100)

In [ ]:
# creating loyalty flag

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# getting results of loyalty flag

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

In [ ]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

In [ ]:
# doing some test's on data through creating subset

test = ords_prods_merge[ords_prods_merge['user_id'] == 15]


In [ ]:
test['order_number'].value_counts()

I had some doubts about one aspect of the data set, but it turns out everything is fine. I can now continue my data analysis as a result.

#### Result of loyalty flag:  

##### Result from subset:  
Regular customer    463844  
Loyal customer      335086  
New customer        201070  
Name: loyalty_flag, dtype: int64    

##### Result from this df  
Regular customer    15876776  
Loyal customer      10284093  
New customer         6243990  
Name: loyalty_flag, dtype: int64  


### 05 customer spending habit based on loyalty flag 

In [ ]:
ords_prods_merge.groupby(['loyalty_flag']).agg({'prices': ['mean', 'min', 'max']})

We can tell from those findings that there is a difference and that new clients appear to purchase more expensive items. Since I still don't know wether to drop those weird pricing numbers or not I will just create a subset excluding these rows and look at it again.

In [ ]:
ords_prods_merge_filter = ords_prods_merge[ords_prods_merge['prices'] <= 50]
# test2.shape

In [ ]:
ords_prods_merge_filter.groupby(['loyalty_flag']).agg({'prices': ['mean', 'min', 'max']})

According to these findings, there is still a difference in the mean customer-purchased product pricing, but it is significantly smaller than before.

### 06 create spending flag
I will keep using test2 for this step because I am still working with the price column.

In [ ]:
# create new column containing average spendeing of customer

ords_prods_merge_filter['avg_spended'] = ords_prods_merge_filter.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
ords_prods_merge_filter.head()

In [ ]:
# create flag

ords_prods_merge_filter.loc[ords_prods_merge_filter['avg_spended'] < 10, 'avg_price_flag'] = 'Low spender'
ords_prods_merge_filter.loc[ords_prods_merge_filter['avg_spended'] >= 10, 'avg_price_flag'] = 'High spender'

In [ ]:
ords_prods_merge_filter['avg_price_flag'].value_counts()

### 07 order frequency flag

In [ ]:
# create new column containing mean  of days_since_prior_order

ords_prods_merge_filter['mean_order_frequency'] = ords_prods_merge_filter.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
ords_prods_merge_filter.head(30)

In [ ]:
# create flag

ords_prods_merge_filter.loc[ords_prods_merge_filter['mean_order_frequency'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge_filter.loc[(ords_prods_merge_filter['mean_order_frequency'] > 10) & (ords_prods_merge_filter['max_order'] <= 20), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge_filter.loc[ords_prods_merge_filter['mean_order_frequency'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [37]:
ords_prods_merge_filter.head(30)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range,busiest_day,busiest_days,busiest_hours,max_order,loyalty_flag,avg_spended,avg_price_flag,mean_order_frequency,order_frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regulary busy days,Regulary busy hours,10,New customer,6.367797,Low spender,20.5,Regular customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Regulary busy hours,10,New customer,6.367797,Low spender,20.5,Regular customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Busiest hours,10,New customer,6.367797,Low spender,20.5,Regular customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Regulary busy hours,10,New customer,6.367797,Low spender,20.5,Regular customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Busiest hours,10,New customer,6.367797,Low spender,20.5,Regular customer
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Regulary busy days,Regulary busy hours,10,New customer,6.367797,Low spender,20.5,Regular customer
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Busiest days,Busiest hours,10,New customer,6.367797,Low spender,20.5,Regular customer
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Mid-range product,Regularly busy,Busiest days,Busiest hours,10,New customer,6.367797,Low spender,20.5,Regular customer
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Mid-range product,Regularly busy,Busiest days,Busiest hours,10,New customer,6.367797,Low spender,20.5,Regular customer
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Regulary busy hours,10,New customer,6.367797,Low spender,20.5,Regular customer


In [ ]:
ords_prods_merge_filter['order_frequency_flag'].value_counts()

In [39]:
ords_prods_merge_filter.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merged_sale_analysis.pkl'))